In [5]:
import os
import tensorflow as tf
from tensorflow import keras

In [6]:
(train_images, train_labels),(test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [7]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model
model = create_model()
model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 608/1000 [=================>............] - ETA: 0s - loss: 1.4546 - accuracy: 0.6036
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 511us/sample - loss: 1.1812 - accuracy: 0.6840 - val_loss: 0.7603 - val_accuracy: 0.7660
Epoch 2/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.4712 - accuracy: 0.8594
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 180us/sample - loss: 0.4335 - accuracy: 0.8750 - val_loss: 0.5987 - val_accuracy: 0.8090
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2665 - accuracy: 0.9234
Epoch 00003: saving model to t

In [9]:
# Create a basic model instance
model = create_model()

# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4028 - accuracy: 0.8720
Restored model, accuracy: 87.20%


In [10]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
# Create a new model instance
model = create_model()

latest = tf.train.latest_checkpoint(checkpoint_dir)

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4880 - accuracy: 0.8740
Restored model, accuracy: 87.40%


In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4880 - accuracy: 0.8740
Restored model, accuracy: 87.40%


In [13]:
# 1 SavedModel format
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
model.save('my_model')



Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 668us/sample - loss: 1.1604 - accuracy: 0.6620
Epoch 2/5
1000/1000 [==============================] - 0s 126us/sample - loss: 0.4213 - accuracy: 0.8740
Epoch 3/5
1000/1000 [==============================] - 0s 126us/sample - loss: 0.2976 - accuracy: 0.9260

In [15]:
new_model = tf.keras.models.load_model('my_model')
new_model.summary()

# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1000/1000 - 0s - loss: 0.4845 - accuracy: 0.8490
Restored model, accuracy: 84.90%
(1000, 10)


In [16]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')


Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 346us/sample - loss: 1.1207 - accuracy: 0.7000
Epoch 2/5
1000/1000 [==============================] - 0s 79us/sample - loss: 0.4141 - accuracy: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 79us/sample - loss: 0.2752 - accuracy: 0.9330
Epoch 4/5
1000/1000 [==============================] - 0s 113us/sample - loss: 0.2041 - accuracy: 0.9470

In [18]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1000/1000 - 0s - loss: 0.4114 - accuracy: 0.8610
Restored model, accuracy: 86.10%
